In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
def format_number(x):
    if isinstance(x, float):
        if x.is_integer():
            return int(x)  # Convert to integer if the float is a whole number
        else:
            return '{:.2f}'.format(x).rstrip('0').rstrip('.')  # Keep up to 2 decimals, remove trailing zeros
    return x

def flatten_json(json_data):
    data = []
    for circuit, circuit_value in json_data.items(): # circuit size
        for benchmark, benchmark_value in circuit_value.items(): # benchmark_name
            for layout, metrics in benchmark_value.items():
                if layout == 'init':
                    continue
                # if 'lookahead_swap' not in metrics:
                #     continue
                basic_swap = metrics.get('basic_swap', np.nan)
                sabre_swap = metrics.get('sabre_swap', np.nan)
                lookahead_swap = metrics.get('lookahead_swap', np.nan)
# Percentage Reduction=( Swap Sabre−Swap Lookahead) / Swap Sabre * 100
                basic_swap_reduction = ((basic_swap - lookahead_swap) / basic_swap * 100) if basic_swap != 0 and basic_swap is not np.nan and lookahead_swap is not np.nan else np.nan
                sabre_swap_reduction = ((sabre_swap - lookahead_swap) / sabre_swap * 100) if sabre_swap != 0 and sabre_swap is not np.nan and lookahead_swap is not np.nan else np.nan
                
                basic_depth = metrics.get('basic_depth', np.nan)
                sabre_depth = metrics.get('sabre_depth', np.nan)
                lookahead_depth = metrics.get('lookahead_depth', np.nan)
                basic_depth_reduction = ((basic_depth - lookahead_depth) / basic_depth * 100) if basic_depth != 0 and basic_depth is not np.nan and lookahead_depth is not np.nan else np.nan
                sabre_depth_reduction = ((sabre_depth - lookahead_depth) / sabre_depth * 100) if sabre_depth != 0 and sabre_depth is not np.nan and lookahead_depth is not np.nan else np.nan

                data.append({
                    "benchmark": benchmark,
                    "size": circuit, # circuit size
                    "gate": benchmark_value['init']['size'], # total gates
                    "depth": benchmark_value['init']['depth'], # circuit depth
                    "layout": layout,
                    "swap basic": basic_swap, # basic swap
                    "swap sabre": sabre_swap, # sabre swap
                    "swap lookahead": lookahead_swap, # lookahead swap
                    "swap delta basic": basic_swap_reduction,
                    "swap delta sabre": sabre_swap_reduction, # difference between sabre and lookahead, if positive then reduced but if negative then larger than sabre
                    "depth basic": basic_depth, # basic depth
                    "depth sabre": sabre_depth, # sabre depth
                    "depth lookahead": lookahead_depth, # lookahead depth
                    "depth delta basic": basic_depth_reduction,
                    "depth delta sabre": sabre_depth_reduction
                })
    return pd.DataFrame(data)


In [3]:
def rearrange_columns(df, first_col_name):
    """
    Reorder the columns of the DataFrame so that the specified column is first.
    
    Parameters:
    - df: The input DataFrame.
    - first_col_name: The name of the column to move to the first position.
    
    Returns:
    - A DataFrame with the specified column moved to the first position.
    """
    # Check if the specified column exists in the DataFrame
    if first_col_name not in df.columns:
        raise ValueError(f"Column '{first_col_name}' does not exist in the DataFrame.")
    
    # Create a list of columns with the specified column first
    columns = [first_col_name] + [col for col in df.columns if col != first_col_name]
    
    # Reorder the DataFrame
    df_reordered = df[columns]
    
    return df_reordered

In [4]:
def open_json_from_file():
    with open("result/benchmarking_FINAL.json") as json_file:
        return json.load(json_file)

dict_benchmark = open_json_from_file()
print(len(dict_benchmark["5"].keys()))

15


$\Delta s_B$

In [5]:
df = flatten_json(dict_benchmark)
df = df.map(format_number)

def rename_columns(df):
    df.columns = [
        "benchmark",
        # "n",
        "g",
        "d",
        "layout",
        r"$s_B$", # basic swap
        r"$s_S$", # sabre swap
        r"$s_L$", # lookahead swap
        # "swap (%)",
        r"$\Delta s_B$", # swap basic reduction
        r"$\Delta s_S$", # swap sabre reduction
        r"$d_B$", # basic depth
        r"$d_S$", # sabre depth
        r"$d_L$", # lookahead depth
        # "d (%)",
        r"$\Delta d_B$",
        r"$\Delta d_S$"
    ]
    return df
# df = rename_columns(df)

print(df)

    benchmark size  gate  depth            layout  swap basic  swap sabre  \
0          dj    5    36     11         full_20_1           0           0   
1          dj    5    36     11         line_20_1          36           6   
2          dj    5    36     11         full_10_2           0           3   
3          dj    5    36     11          full_7_3           0           0   
4          dj    5    36     11          grid_9_2           9           3   
..        ...  ...   ...    ...               ...         ...         ...   
445    wstate   15   253    135          grid_8_3          21           3   
446    wstate   15   253    135         ring_10_2           0          42   
447    wstate   15   253    135          ring_7_3           0          27   
448    wstate   15   253    135  t_horizontal_5_4          63           3   
449    wstate   15   253    135    t_vertical_5_4         126          15   

    swap lookahead swap delta basic swap delta sabre  depth basic  \
0     

In [7]:
def save_latex_to_file(latex, circuit_size, filename=''):
    # with open(f"table/table_{filename}_{circuit_size}.tex", "w") as file:
    with open(f"table/data_{filename}_{circuit_size}_FINAL.tex", "w") as file:
        file.write(latex)

In [8]:
def build_by_benchmark(df, circuit_size):
    df = df[df['size'] == f"{circuit_size}"]
    # df_n_5 = df_n_5.replace('_', r'\_', regex=True)
    df = df.replace('_', '\\_', regex=True)
    df = df.sort_values(by=['gate', 'benchmark'])
    df = df.drop(columns=['size'])
    df = rename_columns(df)
    latex = df.to_latex(index=False, escape=False, longtable=True, caption=f"Additional swap gates and circuit depth, n = {circuit_size}", label=f"benchmark-table-{circuit_size}")
    # latex = latex.replace('_', r'\_').replace('%', r'\%')
    save_latex_to_file(latex, circuit_size, 'benchmark')
    return df

In [9]:
import os

def build_by_benchmark_excel(df, circuit_size):
    df = df[df['size'] == f"{circuit_size}"]
    df = df.sort_values(by=['gate', 'benchmark'])
    # df = df.drop(columns=['size'])
    # df = rename_columns(df)
    # with pd.ExcelWriter("excel/benchmark_layout.xlsx", mode="a" if os.path.exists("excel/benchmark_layout.xlsx") else "w", engine="openpyxl") as writer: # change mode between a = append and w = write
    with pd.ExcelWriter("excel/data_benchmark_FINAL.xlsx", mode="a" if os.path.exists("excel/data_benchmark_FINAL.xlsx") else "w", engine="openpyxl") as writer: # change mode between a = append and w = write
        df.to_excel(writer, sheet_name=f"n = {circuit_size}", index=False)
    return df

In [10]:
df_benchmark_5 = build_by_benchmark(df, 5)
df_benchmark_10 = build_by_benchmark(df, 10)
df_benchmark_15 = build_by_benchmark(df, 15)

/tmp/ipykernel_46646/461077298.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace('_', '\\_', regex=True)


In [11]:
build_by_benchmark_excel(df, 5)
build_by_benchmark_excel(df, 10)
build_by_benchmark_excel(df, 15)

,benchmark,size,gate,depth,layout,swap basic,swap sabre,swap lookahead,swap delta basic,swap delta sabre,depth basic,depth sabre,depth lookahead,depth delta basic,depth delta sabre
310,ghz,15,17,17,full_20_1,0,0,0,nan,nan,17,17,17,0,0
311,ghz,15,17,17,line_20_1,0,27,42,nan,-55.56,17,29,20,-17.65,31.03
312,ghz,15,17,17,full_10_2,0,12,0,nan,100,17,23,17,0,26.09
313,ghz,15,17,17,full_7_3,0,18,0,nan,100,17,23,17,0,26.09
314,ghz,15,17,17,grid_9_2,18,9,30,-66.67,-233.33,35,23,25,28.57,-8.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,portfoliovqe,15,2505,327,grid_9_2,3033,651,1107,63.5,-70.05,2088,798,471,77.44,40.98
346,portfoliovqe,15,2505,327,grid_8_3,4404,720,744,83.11,-3.33,2212,829,429,80.61,48.25
347,portfoliovqe,15,2505,327,ring_10_2,5427,1125,1593,70.65,-41.6,2195,1049,536,75.58,48.9
348,portfoliovqe,15,2505,327,t_horizontal_5_4,5859,963,1047,82.13,-8.72,2288,809,431,81.16,46.72


In [12]:
def build_by_layout(df, circuit_size):
    df = df[df['size'] == f"{circuit_size}"]
    df = df.groupby(by=['layout']).apply(lambda x: x.sort_values(by=['gate', 'benchmark']))
    df = df.replace('_', '\\_', regex=True)
    df = df.drop(columns=['size']) # remove circuit size
    df = rename_columns(df)
    df = rearrange_columns(df, 'layout')
    latex = df.to_latex(index=False, escape=False, longtable=True, caption=f"Additional swap gates and circuit depth, n = {circuit_size}", label=f"benchmark-table-{circuit_size}")
    # latex = latex.replace('_', r'\_').replace('%', r'\%')
    save_latex_to_file(latex, circuit_size, 'layout')
    return df

In [13]:
def build_by_layout_excel(df, circuit_size):
    df = df[df['size'] == f"{circuit_size}"]
    df = df.groupby(by=['layout']).apply(lambda x: x.sort_values(by=['gate', 'benchmark']))
    # df = df.drop(columns=['size']) # remove circuit size
    # df = rename_columns(df)
    df = rearrange_columns(df, 'layout')

    # with pd.ExcelWriter("excel/excel_layout.xlsx", mode="a" if os.path.exists("excel/benchmark_layout.xlsx") else "w", engine="openpyxl") as writer: # change mode between a = append and w = write
    with pd.ExcelWriter("excel/data_layout_FINAL.xlsx", mode="a" if os.path.exists("excel/data_layout_FINAL.xlsx") else "w", engine="openpyxl") as writer: # change mode between a = append and w = write
        df.to_excel(writer, sheet_name=f"n = {circuit_size}", index=False)
    return df

In [14]:
df_layout_5 = build_by_layout(df, 5)
df_layout_10 = build_by_layout(df, 10)
df_layout_15 = build_by_layout(df, 15)

/tmp/ipykernel_46646/2456750932.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(by=['layout']).apply(lambda x: x.sort_values(by=['gate', 'benchmark']))
/tmp/ipykernel_46646/2456750932.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace('_', '\\_', regex=True)
/tmp/ipykernel_46646/2456750932.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas

In [15]:
build_by_layout_excel(df, 5)
build_by_layout_excel(df, 10)
build_by_layout_excel(df, 15)

/tmp/ipykernel_46646/2024238524.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(by=['layout']).apply(lambda x: x.sort_values(by=['gate', 'benchmark']))
/tmp/ipykernel_46646/2024238524.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(by=['layout']).apply(lambda x: x.sort_values(by=['gate', 'benchmark']))
/tmp/ipykernel_46646/2024238524.py:3: DeprecationWarning: DataFram

layout      benchmark size  gate  depth  \
layout                                                                
full_10_2      312       full_10_2            ghz   15    17     17   
               302       full_10_2             dj   15   118     22   
               322       full_10_2     graphstate   15   150     26   
               432       full_10_2            vqe   15   253     31   
               442       full_10_2         wstate   15   253    135   
...                            ...            ...  ...   ...    ...   
t_vertical_5_4 419  t_vertical_5_4      su2random   15   675     81   
               388  t_vertical_5_4            qnn   15   914    158   
               338  t_vertical_5_4  portfolioqaoa   15  1260    192   
               398  t_vertical_5_4         random   15  1992    412   
               349  t_vertical_5_4   portfoliovqe   15  2505    327   

                    swap basic  swap sabre swap lookahead swap delta basic  \
layout                                                                       
full_10_2      312           0          12              0              nan   
               302          66           6              9            86.36   
               322          18           3             15            16.67   
               432           0           6              0              nan   
               442           0           6              0              nan   
...                        ...         ...            ...              ...   
t_vertical_5_4 419        5304        1041           1098             79.3   
               388        3669         618           1077            70.65   
               338        5304         822           1440            72.85   
               398        2475        1239           1800            27.27   
               349        5304        1014           1251            76.41   

                   swap delta sabre  depth basic  depth sabre depth lookahead  \
layout                                                                          
full_10_2      312              100           17           23              17   
               302              -50           95           27              29   
               322             -400           57           26              29   
               432              100           31           40              31   
               442              100          135          138             135   
...                             ...          ...          ...             ...   
t_vertical_5_4 419            -5.48         1962          604             265   
               388           -74.27         1449          547             344   
               338           -75.18         2150          664             430   
               398           -45.28         2366         1243             658   
               349           -23.37         2280          881             456   

                   depth delta basic depth delta sabre  
layout                                                  
full_10_2      312                 0             26.09  
               302             69.47             -7.41  
               322             49.12            -11.54  
               432                 0              22.5  
               442                 0              2.17  
...                              ...               ...  
t_vertical_5_4 419             86.49             56.13  
               388             76.26             37.11  
               338                80             35.24  
               398             72.19             47.06  
               349                80             48.24  

[150 rows x 15 columns]

In [16]:
print("Benchmark [5]\n", df_benchmark_5.value_counts("benchmark"))
print("Benchmark [10]\n", df_benchmark_10.value_counts("benchmark"))
print("Benchmark [15]\n", df_benchmark_15.value_counts("benchmark"))

Benchmark [5]
 benchmark
dj                10
ghz               10
graphstate        10
portfolioqaoa     10
portfoliovqe      10
qaoa              10
qft               10
qftentangled      10
qnn               10
random            10
realamprandom     10
su2random         10
twolocalrandom    10
vqe               10
wstate            10
Name: count, dtype: int64
Benchmark [10]
 benchmark
dj                10
ghz               10
graphstate        10
portfolioqaoa     10
portfoliovqe      10
qaoa              10
qft               10
qftentangled      10
qnn               10
random            10
realamprandom     10
su2random         10
twolocalrandom    10
vqe               10
wstate            10
Name: count, dtype: int64
Benchmark [15]
 benchmark
dj                10
ghz               10
graphstate        10
portfolioqaoa     10
portfoliovqe      10
qaoa              10
qft               10
qftentangled      10
qnn               10
random            10
realamprandom     10
su2random 

In [21]:
df_layout_15

layout      benchmark     g    d s basic s sabre  \
layout                                                                         
full_10_2      292       full_10_2            ghz    17   17       0       6   
               282       full_10_2             dj   118   22      66       9   
               302       full_10_2     graphstate   150   29      30       6   
               389       full_10_2            vqe   253   31       0       6   
               399       full_10_2         wstate   253  135       0      12   
...                            ...            ...   ...  ...     ...     ...   
t_vertical_5_4 380  t_vertical_5_4      su2random   675   81    5304    1086   
               358  t_vertical_5_4            qnn   914  158    3669     600   
               318  t_vertical_5_4  portfolioqaoa  1260  192    5304     879   
               365  t_vertical_5_4         random  1992  412    2475    1203   
               325  t_vertical_5_4   portfoliovqe  2505  327    5304     942   

                    s look d basic d swap  d look  
layout                                             
full_10_2      292       0      17     20      17  
               282       9      95     33      29  
               302      24      51     40      34  
               389       0      31     41      31  
               399       0     135    141     135  
...                    ...     ...    ...     ...  
t_vertical_5_4 380    1098    1962    658     265  
               358    1077    1449    509     344  
               318    1440    2150    641     430  
               365    1800    2366   1214     658  
               325    1251    2280    834     456  

[127 rows x 10 columns]